<a href="https://colab.research.google.com/github/maureen-zhang/maureen-zhang.github.io/blob/main/Addition%20and%20Multiplication%20Calculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Elliptic curv addition algo from Theorem 6.6
All calculation in this file is for elliptic curve modulo some prime p!

Hope this can make your homework easier :)

-Maureen

In [ ]:
# Helper function for modular inverse
def mod_inverse(a, m):
    """
    Return x such that (a * x) % m == 1, or None if gcd(a, m) != 1.
    Implements extended Euclidean algorithm.
    """
    if m <= 0:
        raise ValueError("Modulus m must be a positive integer")

    a %= m
    if a == 0:
        return None

    # r0 = m, r1 = a; t0,t1 track the Bezout coeff for 'a'
    r0, r1 = m, a
    t0, t1 = 0, 1

    while r1 != 0:
        q = r0 // r1
        r0, r1 = r1, r0 - q * r1
        t0, t1 = t1, t0 - q * t1

    # If gcd(a, m) != 1, inverse doesn't exist
    if r0 != 1:
        return None

    # t0 is the inverse modulo m (may be negative)
    return t0 % m

In [ ]:
def elliptic_curve_addition_mod_p(P1, P2, A, B, p):
    """
    Elliptic Curve Addition Algorithm (Theorem 6.6) modulo p

    Args:
        P1: The first point (x1, y1) or 'O' for the point at infinity.
        P2: The second point (x2, y2) or 'O' for the point at infinity.
        A: The coefficient A in the curve equation Y^2 = X^3 + AX + B.
        B: The coefficient B in the curve equation Y^2 = X^3 + AX + B.
        p: The prime modulus.

    Returns:
        The sum of P1 and P2, as a point (x3, y3) or 'O', modulo p.
    """

    # (a) If P1 = O, then P1 + P2 = P2.
    if P1 == 'O':
        return P2

    # (b) Otherwise, if P2 = O, then P1 + P2 = P1.
    if P2 == 'O':
        return P1

    # (c) Otherwise, write P1 = (x1, y1) and P2 = (x2, y2).
    x1, y1 = P1
    x2, y2 = P2

    # (d) If x1 = x2 and y1 = (-y2) % p, then P1 + P2 = O.
    if x1 == x2 and y1 == (-y2) % p:
        return 'O'

    # (e) Otherwise, define lambda
    if P1 != P2:
        # Ensure no division by zero and calculate modular inverse
        numerator = (y2 - y1) % p
        denominator = (x2 - x1) % p
        if denominator == 0:
             return 'O' # Should be handled by case (d) but as a safeguard
        inv_denominator = mod_inverse(denominator, p)
        if inv_denominator is None:
             return 'O' # Should not happen for a prime p unless denominator is 0
        lambda_val = (numerator * inv_denominator) % p
    else:  # P1 == P2
        # Ensure no division by zero and calculate modular inverse
        numerator = (3 * x1**2 + A) % p
        denominator = (2 * y1) % p
        if denominator == 0:
             return 'O' # Point is on the x-axis, addition results in point at infinity
        inv_denominator = mod_inverse(denominator, p)
        if inv_denominator is None:
             return 'O' # Should not happen for a prime p unless denominator is 0
        lambda_val = (numerator * inv_denominator) % p

    # Calculate x3 and y3 modulo p
    x3 = (lambda_val**2 - x1 - x2) % p
    y3 = (lambda_val * (x1 - x3) - y1) % p
    return (x3, y3)

In [82]:
# Example for Y^2 = X^3 + 8 over F_13

# Define the curve parameters and the prime modulus
A = 3
B = 8
p = 13

# Example points on the curve (you would need to verify these points are actually on the curve)
P1 = (1, 5)  # Example point 1
P2 = (1, 8)  # Example point 2
P3 = elliptic_curve_addition_mod_p(P1, P2, A, B, p)
print(f"P1 = {P1}")
print(f"P2 = {P2}")
print(f"P1 + P2 = {P3}")

# Example of adding a point to itself (doubling)
P4 = (2, 3)
P5 = elliptic_curve_addition_mod_p(P4, P4, A, B, p)
print(f"\nP4 = {P4}")
print(f"P4 + P4 = {P5}")


P6 = (9, 6)
P7 = (12, 11) # The negative of P6 modulo p
P8 = elliptic_curve_addition_mod_p(P6, P7, A, B, p)
print(f"\nP6 = {P6}")
print(f"P7 = {P7}")
print(f"P6 + P7 = {P8}")

# Example with the point at infinity
P9 = (1, 3)
P10 = 'O'
P11 = elliptic_curve_addition_mod_p(P9, P10, A, B, p)
print(f"\nP9 = {P9}")
print(f"P10 = {P10}")
print(f"P9 + P10 = {P11}")

P1 = (1, 5)
P2 = (1, 8)
P1 + P2 = O

P4 = (2, 3)
P4 + P4 = (12, 11)

P6 = (9, 6)
P7 = (12, 11)
P6 + P7 = (2, 10)

P9 = (1, 3)
P10 = O
P9 + P10 = (1, 3)


# Elliptic Curve Multiploication
- Double and Add Algorithm

In [83]:

def double_and_add(n, point, A, B, p):
    """
    Implements the double and add algorithm for scalar multiplication
    on an elliptic curve.

    Args:
        n: The scalar (integer) to multiply by.
        point: The elliptic curve point.

    Returns:
        The resulting point after scalar multiplication.

    """#
    R = 'O'
    Q = point
    while n != 0:
        if n % 2 == 1:
            R = elliptic_curve_addition_mod_p(R, Q, A, B, p)
        Q = elliptic_curve_addition_mod_p(Q, Q, A, B, p)
        n = n // 2  # Floor division

    return R


In [84]:
# Test the double_and_add function
scalar = 5  # Example scalar
point_to_multiply = (1, 8) # Example point (must be on the curve)
result_point = double_and_add(scalar, point_to_multiply, 3, 8, 13)
print(f"\n{scalar} * {point_to_multiply} = {result_point}")



R = O
Q = (1, 8)
2
R = (1, 8)
Q = (2, 3)
1
R = (1, 8)
Q = (12, 11)
0

5 * (1, 8) = (12, 2)
